# test embeddings

plot tsne of the embeddings extracted by the CNN model trained on the dataset.

-> write a module for input generation. given the folder and file names get NxTxM matrix
-> each data point, load model in eval mode. 
-> forward the given 

In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision.transforms as transforms
import sys
from ret_benchmark.config import cfg
from ret_benchmark.modeling import build_model
import os
import numpy as np
import re
from sklearn.manifold import TSNE
import pickle
import matplotlib.pyplot as plt
from itertools import combinations

os.environ['CUDA_VISIBLE_DEVICES'] = '3' # gpu to run the code
sys.path.insert(1, '/data/home/shruti/voxceleb/motion_signature/research-ms-loss-master/')
config_path = 'configs/directms_aus190.yaml'
cache_emb_file = 'directms_aus190_withFaceSwap' # this file is cache for fabnet metric learning embeddings on leader data


In [2]:
# get all the embedding
def load_obj(fldr, name ):
    if not os.path.exists(os.path.join(fldr, name + '.pkl')):
        return None
    with open(os.path.join(fldr, name + '.pkl'), 'rb') as f:
        return pickle.load(f)

# save the obj
def save_obj(obj, fldr, name ):
    os.makedirs(fldr, exist_ok=True)
    with open(os.path.join(fldr, name + '.pkl'), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

## load model

In [3]:
def load_model(cfgpath):
    
    cfg.merge_from_file(cfgpath)
    
    # load the model
    model = build_model(cfg)
    device = torch.device(cfg.MODEL.DEVICE)
    model.to(device)
    model.eval()
    return model

In [4]:
model = load_model(config_path)

Resuming from model ...


## load data and labels

In [5]:
subfldrs = np.array(['bo_UWfake', 'diff_test', 'dt_imposter', 'hc_imposter', 
                     'bs_imposter', 'ew_imposter', 'bo_imposter', 'trump_fake'])

def get_fabnet_ldrs_data():
        
    # read mean and std files
    means = np.reshape(np.load(cfg.INPUT.MEAN), (1, -1))
    stds = np.reshape(np.load(cfg.INPUT.STD), (1, -1))

    # the test files of leaders
    root = os.path.dirname(cfg.DATA.TEST_IMG_SOURCE)
    path_list = []
    label_list = []
    print(cfg.DATA.TRAIN_IMG_SOURCE)
    with open(cfg.DATA.TRAIN_IMG_SOURCE, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
            
    print(cfg.DATA.TEST_IMG_SOURCE)
    with open(cfg.DATA.TEST_IMG_SOURCE, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
    
    test_file = cfg.DATA.TEST_IMG_SOURCE.replace('_val', '_test')
    print(test_file)
    with open(test_file, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
    
    # add the files from 
    ofset = np.max(label_list) + 1
    for j in range(len(subfldrs)):
        _path = [os.path.join(subfldrs[j], f) for f in os.listdir(os.path.join(root, subfldrs[j])) if f.endswith('.npy')]
        _label = list(np.zeros((len(_path), ), dtype='int')+ofset+j)
        path_list.extend(_path)
        label_list.extend(_label)
    
    data = {}
    data['mean'] = means
    data['std'] = stds
    data['root'] = root
    data['path_list'] = path_list
    data['label_list'] = label_list
    
    return data

def get_vgg_ldrs_data():
        
    # read mean and std files
    means = np.reshape(np.load(cfg.INPUT.MEAN), (1, -1))
    stds = np.reshape(np.load(cfg.INPUT.STD), (1, -1))

    # the test files of leaders
    root = os.path.dirname(cfg.DATA.TEST_IMG_SOURCE)
    path_list = []
    label_list = []
    print(cfg.DATA.TRAIN_IMG_SOURCE)
    with open(cfg.DATA.TRAIN_IMG_SOURCE, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
            
    print(cfg.DATA.TEST_IMG_SOURCE)
    with open(cfg.DATA.TEST_IMG_SOURCE, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
    
    test_file = cfg.DATA.TEST_IMG_SOURCE.replace('_val', '_test')
    print(test_file)
    with open(test_file, 'r') as f:
        for line in f:
            _path, _label = re.split(r",| ", line.strip())
            path_list.extend([_path])
            label_list.extend([int(_label)])
    
    # add the files from 
    ofset = np.max(label_list) + 1
    for j in range(len(subfldrs)):
        _path = [f for f in os.listdir(root) if f.endswith('.npy') and f[:len(subfldrs[j])+1] == subfldrs[j]+'_']
        _label = list(np.zeros((len(_path), ), dtype='int')+ofset+j)
        path_list.extend(_path)
        label_list.extend(_label)
    
    data = {}
    data['mean'] = means
    data['std'] = stds
    data['root'] = root
    data['path_list'] = path_list
    data['label_list'] = label_list
    
    return data

In [6]:
#data_files = get_fabnet_ldrs_data()
data_files = get_vgg_ldrs_data()
print(len(data_files['path_list']))

/data/home/shruti/voxceleb/aus/leaders/leaders_100_train.txt
/data/home/shruti/voxceleb/aus/leaders/leaders_100_val.txt
/data/home/shruti/voxceleb/aus/leaders/leaders_100_test.txt
4710


# get metric embedding

for every video file:
    1) load the npy
    2) for every 150 window with 5 frame step, pass it through the model and get the embedding
    3) in the save the embedding and the label in a dictionary
    4) save the embeddings 

In [7]:

feat_pair = list(combinations(range(20), 2))
feat_pair = np.array([[j[0], j[1]] for j in feat_pair])
def aus190_lbl(in_feat):
    
    aus = in_feat[:, :20]
    corr = np.corrcoef(aus[:, feat_pair[:, 0]].T, 
                       aus[:, feat_pair[:, 1]].T)[0:len(feat_pair), 
                                                  len(feat_pair):].diagonal()
        
    if np.sum(np.isnan(corr))>0:
        return None
    
    return np.reshape(corr, (1, -1)).copy()


def FabNet(in_feat):
    
    fab_emb = in_feat/np.linalg.norm(in_feat, axis=1, keepdims=True)
    return (in_feat-data_files['mean'])/data_files['std']

# dataset loader
data_loader_dict = {'aus190_lbl': aus190_lbl, 'FabNet': FabNet}

In [8]:
def feature_extract(infile):
    
    st = 5; emb_sz = cfg.MODEL.HEAD.DIM
    try:
        fab_emb = np.load(infile)
    except Exception as e:
        print(e)
        return None
    
    # if not enough time window
    if len(fab_emb)<cfg.INPUT.FRAME_LENGTH:
        return None
    
    if len(fab_emb.shape)>2:
        fab_emb = fab_emb[:, :, 0].copy()
    
    feat_rng = np.arange(0, len(fab_emb)-cfg.INPUT.FRAME_LENGTH+1, st)
    out_feat = np.zeros((len(feat_rng), emb_sz)) + np.nan
    
    print(f' {out_feat.shape}')
    for i in range(len(feat_rng)):
        cur_data = data_loader_dict[cfg.INPUT.DATA_LOADER](fab_emb[feat_rng[i]:feat_rng[i]+cfg.INPUT.FRAME_LENGTH, :])
        
        if cur_data is None:
            continue
        
        #transform
        cur_data = np.reshape(cur_data, (1, 1, cur_data.shape[0], cur_data.shape[1]))
        tensor_data = torch.from_numpy(cur_data).cuda().float()

        # pass the model
        with torch.no_grad():
            out_feat[i, :] = model(tensor_data).data.cpu().numpy().ravel()
    
    # remove nans
    out_feat = out_feat[np.sum(np.isnan(out_feat), axis=1)<1, :].copy()
    return out_feat

In [9]:
int2str_lbl = {0:'bo', 1:'br', 2:'bs', 3:'cb', 4:'dt_week', 4:'dt_rndm', 5:'ew', 6:'hc', 7:'jb', 8:'kh', 9:'pb', 
               10:'bo_faceswap', 11:'dt_faceswap', 12:'hc_faceswap', 13:'bs_faceswap',
               14:'ew_faceswap', 15:'bo_UWfake', 16:'diff_test', 17:'dt_imposter', 18:'hc_imposter', 19:'bs_imposter', 20:'ew_imposter', 
               21:'bo_imposter', 22:'trump_fake'}

for lbl in int2str_lbl.keys():
    
    # for every embedding file
    cur_dict = load_obj( f'obj/{cache_emb_file}', f'{cfg.INPUT.FRAME_LENGTH}_{int2str_lbl[lbl]}')    
    if cur_dict is None:

        cur_dict = {}
        cur_files = np.array(data_files['path_list'])[np.array(data_files['label_list'])==lbl]
        for i in range(len(cur_files)): #

            print(cur_files[i], end=' ')
            # 
            cur_feat = feature_extract(os.path.join(data_files['root'], cur_files[i]))
            if cur_feat is not None:
                cur_dict[cur_files[i]] = cur_feat

        save_obj(cur_dict,  f'obj/{cache_emb_file}', f'{cfg.INPUT.FRAME_LENGTH}_{int2str_lbl[lbl]}')

bo_--QxAtXZJpQ_0.npy  (272, 190)


/data/opt/voxceleb/lib/python3.6/site-packages/numpy/lib/function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/data/opt/voxceleb/lib/python3.6/site-packages/numpy/lib/function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


bo_--QxAtXZJpQ_1.npy  (306, 190)
bo_--QxAtXZJpQ_2.npy  (305, 190)
bo_--QxAtXZJpQ_3.npy  (305, 190)
bo_-9mSTKUWS7U_0.npy  (252, 190)
bo_-9mSTKUWS7U_1.npy  (310, 190)
bo_-9mSTKUWS7U_2.npy  (308, 190)
bo_-9mSTKUWS7U_3.npy  (310, 190)
bo_-9mSTKUWS7U_4.npy  (9, 190)
bo_-LhfOv1-wOQ_0.npy  (252, 190)
bo_-LhfOv1-wOQ_1.npy  (310, 190)
bo_-LhfOv1-wOQ_2.npy  (310, 190)
bo_-LhfOv1-wOQ_3.npy  (117, 190)
bo_-ZJvRzOdSCw_0.npy  (277, 190)
bo_-ZJvRzOdSCw_1.npy  (303, 190)
bo_-ZJvRzOdSCw_2.npy  (305, 190)
bo_-ZJvRzOdSCw_3.npy  (119, 190)
bo_-my9hVoGv2M_0.npy  (275, 190)
bo_-my9hVoGv2M_1.npy  (309, 190)
bo_-my9hVoGv2M_2.npy  (310, 190)
bo_-my9hVoGv2M_3.npy  (310, 190)
bo_-my9hVoGv2M_4.npy  (305, 190)
bo_-my9hVoGv2M_5.npy  (124, 190)
bo_0SaVqB0w718_0.npy  (251, 190)
bo_0SaVqB0w718_1.npy  (310, 190)
bo_0SaVqB0w718_2.npy  (309, 190)
bo_0SaVqB0w718_3.npy  (264, 190)
bo_0T_Jv4Kbgek_0.npy  (247, 190)
bo_0T_Jv4Kbgek_1.npy  (310, 190)
bo_0T_Jv4Kbgek_2.npy  (302, 190)
bo_0_9ue3Tx8Jk_0.npy  (277, 190)
bo_0_9ue3Tx8

bo_CtSHDC-bsu4_3.npy  (310, 190)
bo_CtSHDC-bsu4_4.npy  (51, 190)
bo_DAi7--S3Y5k_0.npy  (274, 190)
bo_DAi7--S3Y5k_1.npy  (296, 190)
bo_DAi7--S3Y5k_2.npy  (210, 190)
bo_Dt46A8dvRSs_0.npy  (249, 190)
bo_Dt46A8dvRSs_1.npy  (309, 190)
bo_Dt46A8dvRSs_2.npy  (308, 190)
bo_Dt46A8dvRSs_3.npy  (109, 190)
bo_Dxsdp3r5Dek_0.npy  (275, 190)
bo_Dxsdp3r5Dek_1.npy  (305, 190)
bo_Dxsdp3r5Dek_2.npy  (310, 190)
bo_Dxsdp3r5Dek_3.npy  (125, 190)
bo_E3gfMumXCjI_0.npy  (246, 190)
bo_E3gfMumXCjI_1.npy  (305, 190)
bo_E3gfMumXCjI_2.npy  (309, 190)
bo_E3gfMumXCjI_3.npy  (264, 190)
bo_EAZIHIiuhrc_0.npy  (252, 190)
bo_EAZIHIiuhrc_1.npy  (310, 190)
bo_EAZIHIiuhrc_2.npy  (311, 190)
bo_EAZIHIiuhrc_3.npy  (95, 190)
bo_EMyqZ8lGgXY_0.npy  (246, 190)
bo_EMyqZ8lGgXY_1.npy  (310, 190)
bo_EMyqZ8lGgXY_2.npy  (304, 190)
bo_EMyqZ8lGgXY_3.npy  (149, 190)
bo_Eh4f3it89BU_0.npy  (276, 190)
bo_Eh4f3it89BU_1.npy  (310, 190)
bo_Eh4f3it89BU_2.npy  (306, 190)
bo_Eh4f3it89BU_3.npy  (308, 190)
bo_Eh4f3it89BU_4.npy  (31, 190)
bo_FDa3Gmq36w

bo_TVF8e5AuY10_2.npy  (308, 190)
bo_TVF8e5AuY10_3.npy  (288, 190)
bo_Tpaj9Sm7i7I_0.npy  (249, 190)
bo_Tpaj9Sm7i7I_1.npy  (309, 190)
bo_Tpaj9Sm7i7I_2.npy  (310, 190)
bo_Tpaj9Sm7i7I_3.npy  (309, 190)
bo_Tpaj9Sm7i7I_4.npy  (46, 190)
bo_V1fA9-SryzY_0.npy  (276, 190)
bo_V1fA9-SryzY_1.npy  (310, 190)
bo_V1fA9-SryzY_2.npy  (308, 190)
bo_V1fA9-SryzY_3.npy  (284, 190)
bo_V5_vUE9FlXk_0.npy  (270, 190)
bo_V5_vUE9FlXk_1.npy  (305, 190)
bo_V5_vUE9FlXk_2.npy  (243, 190)
bo_VWc_aHqaPeE_0.npy  (246, 190)
bo_VWc_aHqaPeE_1.npy  (308, 190)
bo_VWc_aHqaPeE_2.npy  (310, 190)
bo_VWc_aHqaPeE_3.npy  (138, 190)
bo_Vd7aR5PMpuQ_0.npy  (276, 190)
bo_Vd7aR5PMpuQ_1.npy  (309, 190)
bo_Vd7aR5PMpuQ_2.npy  (309, 190)
bo_Vd7aR5PMpuQ_3.npy  (311, 190)
bo_Vd7aR5PMpuQ_4.npy  (27, 190)
bo_WQM6VG5F3Ek_0.npy  (246, 190)
bo_WQM6VG5F3Ek_1.npy  (309, 190)
bo_WQM6VG5F3Ek_2.npy  (308, 190)
bo_WsypI6avDEc_0.npy  (251, 190)
bo_WsypI6avDEc_1.npy  (309, 190)
bo_WsypI6avDEc_2.npy  (310, 190)
bo_WsypI6avDEc_3.npy  (307, 190)
bo_WsypI6avD

...

jb_sqs5TKH89pE_0_0.npy  (178, 190)
jb_sqs5TKH89pE_1_0.npy  (194, 190)
jb_sqs5TKH89pE_2_0.npy  (160, 190)
jb_sqs5TKH89pE_3_0.npy  (307, 190)
jb_sqs5TKH89pE_3_1.npy  (307, 190)
jb_sqs5TKH89pE_4_0.npy  (141, 190)
jb_sqs5TKH89pE_5_0.npy  (59, 190)
jb_vQrMvnw1xjw_1_0.npy  (295, 190)
jb_vQrMvnw1xjw_2_0.npy  (218, 190)
jb_vQrMvnw1xjw_2_1.npy  (217, 190)
jb_vQrMvnw1xjw_3_0.npy  (210, 190)
jb_vQrMvnw1xjw_3_1.npy  (209, 190)
jb_vQrMvnw1xjw_4_0.npy  (222, 190)
jb_vQrMvnw1xjw_4_1.npy  (222, 190)
jb_vQrMvnw1xjw_5_0.npy  (71, 190)
jb_wcyXQJ2Tf4E_0_0.npy  (262, 190)
jb_wcyXQJ2Tf4E_1_0.npy  (220, 190)
jb_wcyXQJ2Tf4E_2_0.npy  (274, 190)
jb_wcyXQJ2Tf4E_2_1.npy  (274, 190)
jb_wcyXQJ2Tf4E_2_2.npy  (274, 190)
jb_wiveFTgYFCo_0_0.npy  (330, 190)
jb_wiveFTgYFCo_0_1.npy  (332, 190)
jb_wiveFTgYFCo_0_2.npy  (332, 190)
jb_wiveFTgYFCo_0_3.npy  (332, 190)
jb_wiveFTgYFCo_0_4.npy  (332, 190)
jb_wiveFTgYFCo_0_5.npy  (332, 190)
jb_xZVVB0pm_tY_0_0.npy  (250, 190)
jb_xZVVB0pm_tY_1_0.npy  (141, 190)
jb_xZVVB0pm_tY_2_0.npy

pb_4Re5OyMdtQE_10_0.npy  (157, 190)
pb_4Re5OyMdtQE_11_0.npy  (254, 190)
pb_4Re5OyMdtQE_12_0.npy  (233, 190)
pb_4Re5OyMdtQE_14_0.npy  (251, 190)
pb_4Re5OyMdtQE_15_0.npy  (296, 190)
pb_4Re5OyMdtQE_16_0.npy  (229, 190)
pb_4Re5OyMdtQE_19_0.npy  (170, 190)
pb_4Re5OyMdtQE_19_1.npy  (170, 190)
pb_4Re5OyMdtQE_23_0.npy  (305, 190)
pb_4Re5OyMdtQE_6_0.npy  (53, 190)
pb_4Re5OyMdtQE_7_0.npy  (143, 190)
pb_4Re5OyMdtQE_8_0.npy  (41, 190)
pb_6jFKWokwE6g_10_0.npy  (314, 190)
pb_6jFKWokwE6g_12_0.npy  (47, 190)
pb_6jFKWokwE6g_14_0.npy  (263, 190)
pb_6jFKWokwE6g_16_0.npy  (242, 190)
pb_6jFKWokwE6g_16_1.npy  (242, 190)
pb_6jFKWokwE6g_4_0.npy  (53, 190)
pb_6jFKWokwE6g_5_0.npy  (83, 190)
pb_6jFKWokwE6g_6_0.npy  (164, 190)
pb_6jFKWokwE6g_8_0.npy  (178, 190)
pb_6jFKWokwE6g_9_0.npy  (62, 190)
pb_8ZtZNo3OlMk_1_0.npy  (319, 190)
pb_8ZtZNo3OlMk_1_1.npy  (319, 190)
pb_8ZtZNo3OlMk_2_0.npy  (255, 190)
pb_9LUI4EOGMq4_0_0.npy  (214, 190)
pb_9LUI4EOGMq4_0_1.npy  (214, 190)
pb_9LUI4EOGMq4_11_0.npy  (86, 190)
pb_9LUI4EOGM

dt_faceswap_02_output.npy  (149, 190)
dt_faceswap_03_output_0.npy  (265, 190)
dt_faceswap_03_output_1.npy  (265, 190)
dt_faceswap_03_output_2.npy  (264, 190)
dt_faceswap_04_output.npy  (197, 190)
dt_faceswap_05_output.npy  (65, 190)
dt_faceswap_06_output.npy  (47, 190)
dt_faceswap_07_output.npy  (131, 190)
dt_faceswap_08_output.npy  (137, 190)
dt_faceswap_09_output.npy  (17, 190)
dt_faceswap_10_output.npy  (41, 190)
dt_faceswap_11_output.npy  (119, 190)
dt_faceswap_12_output.npy  (41, 190)
dt_faceswap_13_output.npy  (5, 190)
dt_faceswap_14_output.npy  (59, 190)
dt_faceswap_15_output.npy  (119, 190)
dt_faceswap_16_output.npy  (113, 190)
dt_faceswap_17_output.npy  (53, 190)
dt_faceswap_18_output.npy  (113, 190)
dt_faceswap_19_output.npy  (65, 190)
dt_faceswap_20_output.npy  (107, 190)
dt_faceswap_21_output.npy  (251, 190)
dt_faceswap_22_output.npy  (101, 190)
dt_faceswap_23_output.npy  (71, 190)
dt_faceswap_24_output.npy  (238, 190)
hc_faceswap_01_output.npy  (101, 190)
hc_faceswap_02_ou

bo_UWfake_Obama_fake_uw_4.npy  (175, 190)
bo_UWfake_debug_yfanOzK0v0s}}00_25GOnaY8ZCY_all_0.npy  (281, 190)
bo_UWfake_debug_nIxM8rL5GVE}}00_25GOnaY8ZCY_data10_0.npy  (180, 190)
bo_UWfake_debug_rGLahI7B9Ws}}00_25GOnaY8ZCY_0.npy  (203, 190)
bo_UWfake_debug_7XGi3FGVmA0}}00_25GOnaY8ZCY_0.npy  (111, 190)
bo_UWfake_debug_3vPdtajOJfw}}01_3vPdtajOJfw_1.npy  (224, 190)
bo_UWfake_debug_test}}00_2AFpAATHXtc_0.npy  (19, 190)
bo_UWfake_debug_nIxM8rL5GVE}}00_25GOnaY8ZCY_data1_0.npy  (180, 190)
bo_UWfake_debug_nIxM8rL5GVE}}00_k4OZOTaf3lk_1.npy  (179, 190)
bo_UWfake_debug_cNVzN62l0Yg}}00_3vPdtajOJfw_1.npy  (160, 190)
bo_UWfake_debug_cNVzN62l0Yg}}00_25GOnaY8ZCY_0.npy  (161, 190)
bo_UWfake_Obama_fake_uw_0.npy  (44, 190)
bo_UWfake_debug_cNVzN62l0Yg}}00_3vPdtajOJfw_refined_1.npy  (160, 190)
bo_UWfake_debug_7G5kMmnAp_8}}00_25GOnaY8ZCY_1.npy  (184, 190)
bo_UWfake_Obama_fake_uw_fulldataset_0.npy  (20, 190)
bo_UWfake_debug_nIxM8rL5GVE}}00_25GOnaY8ZCY_data50_0.npy  (180, 190)
bo_UWfake_debug_nIxM8rL5GVE}}00_3v

diff_test_ukjUkWWRs9w.npy  (99, 190)
diff_test_Z2aWbbwo6I0.npy  (110, 190)
diff_test_jjOsejIqlR0.npy  (100, 190)
diff_test_ca45Slea2vo.npy  (50, 190)
diff_test_xOHUoBze-CU.npy  (92, 190)
diff_test_kkOJhJ6yj6I.npy  (90, 190)
diff_test_dY2Tk6DUXMg.npy  (152, 190)
diff_test_bOZOLDoDFSs.npy  (80, 190)
diff_test_wxR-NKILUT4.npy  (85, 190)
diff_test_gbQK8ucIcAw.npy  (58, 190)
diff_test_rai6unYUpiM.npy  (78, 190)
diff_test_5b96N4ciJRY.npy  (105, 190)
diff_test_9jGK1fRtNI0.npy  (52, 190)
diff_test_zR3A0fzf1Kg.npy  (64, 190)
diff_test_xQHrfDT6XJ0.npy  (49, 190)
diff_test_1-28pATD3xM.npy  (69, 190)
diff_test_xJwHH4yVaYs.npy  (130, 190)
diff_test_Tn03Kw2i4Lw.npy  (80, 190)
diff_test_z3NNFRvZgNs.npy  (121, 190)
diff_test_t2T_pvkLm3s.npy  (109, 190)
diff_test_G-DBOlOFzBs.npy  (75, 190)
diff_test_NS1X-mcitLI.npy  (196, 190)
diff_test_qpBoMZ8mAiQ.npy  (85, 190)
diff_test_pUbvK1C3dmI.npy  (70, 190)
diff_test_t2XY5TCTdUY.npy  (45, 190)
diff_test_GunF3fefH0M.npy  (44, 190)
diff_test_Zh54aq6BiBY.npy  (72